<a href="https://colab.research.google.com/github/XueyanZhang/MachineLearningCompilation/blob/master/MLC_TensorIR_Excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!python3 -m  pip install mlc-ai-nightly -f https://mlc.ai/wheels

import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://mlc.ai/wheels


# Write Tensor IR

## 1. element-wise add



### high-level numpy addition

In [28]:
a = np.arange(16).reshape(4, 4)
b = np.arange(16, 0, -1).reshape(4, 4)

print(a)
print(type(a))
print()
print(b)
print(type(b))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
<class 'numpy.ndarray'>

[[16 15 14 13]
 [12 11 10  9]
 [ 8  7  6  5]
 [ 4  3  2  1]]
<class 'numpy.ndarray'>


In [29]:
# numpy addition
c_np = a + b
c_np

array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

### low-level numpy addition

In [30]:
# low level numpy (w/ loops)
def lnumpy_add(a: np.ndarray, b: np.ndarray, c: np.ndarray):
    for i in range(4):
        for j in range(4):
            c[i, j] = a[i, j] + b[i, j]

c_np_low = np.empty((4, 4), dtype=np.int64)
lnumpy_add(a, b, c_np_low)
c_np_low

array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

### Tensor IR addition

In [31]:
# tensor ir
@tvm.script.ir_module
class MyAdd:
    @T.prim_func
    def add(A: T.Buffer((4, 4), "int64"),
            B: T.Buffer((4, 4), "int64"),
            C: T.Buffer((4, 4), "int64")):
        T.func_attr({"global_symbol": "add"})
        for i, j in T.grid(4, 4):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                C[vi, vj] = A[vi, vj] + B[vi, vj]

In [32]:
# build and run
rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.empty((4, 4), dtype="int64")

rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)
c_tvm


<tvm.nd.NDArray shape=(4, 4), cpu(0)>
array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

## 2. broadcast add

In [33]:
a = np.arange(16).reshape(4, 4)
b = np.arange(4, 0, -1).reshape(4)

print(a)
print()
print(b)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]

[4 3 2 1]


In [34]:
# high level numpy
c_np = a + b
c_np

array([[ 4,  4,  4,  4],
       [ 8,  8,  8,  8],
       [12, 12, 12, 12],
       [16, 16, 16, 16]])

In [35]:
# low level numpy
def lnumpy_broadcastAdd(a: np.ndarray, b: np.ndarray, c: np.ndarray):
    for i in range(4):
        for j in range(4):
            c[i, j] = a[i, j] + b[j]

c_np_low = np.empty((4, 4), dtype=np.int64)
lnumpy_broadcastAdd(a, b, c_np_low)
c_np_low

array([[ 4,  4,  4,  4],
       [ 8,  8,  8,  8],
       [12, 12, 12, 12],
       [16, 16, 16, 16]])

In [36]:
# tensor ir
@tvm.script.ir_module
class MyBCSTAdd:
    @T.prim_func
    def bcstadd(A: T.Buffer((4, 4), "int64"),
                B: T.Buffer((4), "int64"),
                C: T.Buffer((4, 4), "int64")):
        T.func_attr({"global_symbol": "bcstadd"})
        for i, j in T.grid(4, 4):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                C[vi, vj] = A[vi, vj] + B[vj]

In [37]:
rt_lib = tvm.build(MyBCSTAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.empty((4, 4), dtype="int64")

rt_lib["bcstadd"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)
c_tvm

<tvm.nd.NDArray shape=(4, 4), cpu(0)>
array([[ 4,  4,  4,  4],
       [ 8,  8,  8,  8],
       [12, 12, 12, 12],
       [16, 16, 16, 16]])

## 3. 2D convolution

Here is the mathematical definition of convolution with NCHW layout:
 
C
o
n
v
[
b
,
k
,
i
,
j
]
=
∑_
(d
i
,
d
j
,
q)
A
[
b
,
q
,
s
t
r
i
d
e
s
∗
i
+
d
i
,
s
t
r
i
d
e
s
∗
j
+
d
j
]
∗
W
[
k
,
q
,
d
i
,
d
j
]

, where, A is the input tensor, W is the weight tensor, b is the batch index, k is the out channels, i and j are indices for image hight and width, di and dj are the indices of the weight, q is the input channel, and strides is the stride of the filter window.

In [56]:
# string = 1, padding = 0
N, CI, H, W, CO, K = 1, 1, 8, 8, 2, 3
OUT_H, OUT_W = H - K + 1, W - K + 1
data = np.arange(N * CI * H * W).reshape(N, CI, H, W)
weight = np.arange(CO * CI * K * K).reshape(CO, CI, K, K)

# print(data)
# print()
# print(weight)

In [58]:
# torch version
import torch

data_torch = torch.Tensor(data)
weight_torch = torch.Tensor(weight)
conv_torch = torch.nn.functional.conv2d(data_torch, weight_torch)
conv_torch = conv_torch.numpy().astype(np.int64)
conv_torch

array([[[[ 474,  510,  546,  582,  618,  654],
         [ 762,  798,  834,  870,  906,  942],
         [1050, 1086, 1122, 1158, 1194, 1230],
         [1338, 1374, 1410, 1446, 1482, 1518],
         [1626, 1662, 1698, 1734, 1770, 1806],
         [1914, 1950, 1986, 2022, 2058, 2094]],

        [[1203, 1320, 1437, 1554, 1671, 1788],
         [2139, 2256, 2373, 2490, 2607, 2724],
         [3075, 3192, 3309, 3426, 3543, 3660],
         [4011, 4128, 4245, 4362, 4479, 4596],
         [4947, 5064, 5181, 5298, 5415, 5532],
         [5883, 6000, 6117, 6234, 6351, 6468]]]])

In [62]:
# low level numpy
def lnumpy_conv2d(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    for b in range(N):
        for c_out in range(CO):
            for h_out in range(OUT_H):
                for w_out in range(OUT_W):
                    # compute the output tensor at (b, c_out, h_out, w_out)
                    for c_in in range(CI):
                        for kh in range(K):
                            for kw in range(K):
                                h_in = h_out + kh
                                w_in = w_out + kw
                                C[b, c_out, h_out, w_out] += (
                                    A[b, c_in, h_in, w_in] * B[c_out, c_in, kh, kw]
                                )

conv_lnumpy = np.zeros((N, CO, OUT_H, OUT_W), dtype=np.int64)
lnumpy_conv2d(data, weight, conv_lnumpy)
conv_lnumpy

array([[[[ 474,  510,  546,  582,  618,  654],
         [ 762,  798,  834,  870,  906,  942],
         [1050, 1086, 1122, 1158, 1194, 1230],
         [1338, 1374, 1410, 1446, 1482, 1518],
         [1626, 1662, 1698, 1734, 1770, 1806],
         [1914, 1950, 1986, 2022, 2058, 2094]],

        [[1203, 1320, 1437, 1554, 1671, 1788],
         [2139, 2256, 2373, 2490, 2607, 2724],
         [3075, 3192, 3309, 3426, 3543, 3660],
         [4011, 4128, 4245, 4362, 4479, 4596],
         [4947, 5064, 5181, 5298, 5415, 5532],
         [5883, 6000, 6117, 6234, 6351, 6468]]]])

In [63]:
# tensor ir version
@tvm.script.ir_module
class MyConv2d:
  @T.prim_func
  def conv2d(data: T.Buffer((N, CI, H, W), "int64"), 
             weight: T.Buffer((CO, CI, K, K), "int64"),
             conv: T.Buffer((N, CO, OUT_H, OUT_W), "int64")):
    T.func_attr({"global_symbol": "conv2d", "tir.noalias": True})
    for n, co, ho, wo, ci, hk, hw in T.grid(N, CO, OUT_H, OUT_W, CI, K, K):
        with T.block("C"):
            b, c_out, h_out, w_out, c_in, kh, kw = T.axis.remap("SSSSRRR", [n, co, ho, wo, ci, hk, hw])
            with T.init():
                conv[b, c_out, h_out, w_out] = T.int64(0)
            h_in = h_out + kh
            w_in = w_out + kw
            conv[b, c_out, h_out, w_out] += data[b, c_in, h_in, w_in] * weight[c_out, c_in, kh, kw]


In [65]:
rt_lib = tvm.build(MyConv2d, target="llvm")
data_tvm = tvm.nd.array(data)
weight_tvm = tvm.nd.array(weight)
conv_tvm = tvm.nd.array(np.empty((N, CO, OUT_H, OUT_W), dtype=np.int64))
rt_lib["conv2d"](data_tvm, weight_tvm, conv_tvm)
np.testing.assert_allclose(conv_tvm.numpy(), conv_torch, rtol=1e-5)
conv_tvm

<tvm.nd.NDArray shape=(1, 2, 6, 6), cpu(0)>
array([[[[ 474,  510,  546,  582,  618,  654],
         [ 762,  798,  834,  870,  906,  942],
         [1050, 1086, 1122, 1158, 1194, 1230],
         [1338, 1374, 1410, 1446, 1482, 1518],
         [1626, 1662, 1698, 1734, 1770, 1806],
         [1914, 1950, 1986, 2022, 2058, 2094]],

        [[1203, 1320, 1437, 1554, 1671, 1788],
         [2139, 2256, 2373, 2490, 2607, 2724],
         [3075, 3192, 3309, 3426, 3543, 3660],
         [4011, 4128, 4245, 4362, 4479, 4596],
         [4947, 5064, 5181, 5298, 5415, 5532],
         [5883, 6000, 6117, 6234, 6351, 6468]]]])

# Transform Tensor IR